In [1]:
from CR_LO import Loader
from CR_FE_B import FExtractor
from TR_B import Trainer
from TE_B import Tester

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from UCroma import PretrainedCROMA
import copy

import pandas as pd
import os
import pickle
    
sample_size = 120
batch_size = 16

Croma = PretrainedCROMA(pretrained_path='CR.pt', size='base', modality='both', image_resolution=sample_size)
num_features = Croma(SAR_images=torch.randn(1, 2, sample_size, sample_size), 
                     optical_images=torch.randn(1, 12, sample_size, sample_size))['joint_GAP'].shape[1]
head = nn.Sequential(nn.Linear(num_features, 1000), nn.ReLU(), nn.Linear(1000, 1000), nn.ReLU(), nn.Linear(1000, 2))

/home/stagiaire/miniconda3/envs/croma/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/stagiaire/miniconda3/envs/croma/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Initializing SAR encoder
Initializing optical encoder
Initializing joint SAR-optical encoder


In [2]:
it = 10
metrics = []

data = {}
for j in range(1, 5):
    root_dir = f"/home/stagiaire/D/D/patchs/{j}n"  
    for folder in os.listdir(root_dir):
        root_folder = os.path.join(root_dir, folder)
        files = os.listdir(root_folder)
        for file in files:
            n = file.split('.')[0]
            data[n] = []
global_list = pd.DataFrame(list(data.items()), columns=['Id', 'List'])
global_list = global_list.set_index('Id')

processor = Loader(opt_root_dir=f"/home/stagiaire/D/D/patchs/{3}n", sar_root_dir=f"/home/stagiaire/D/D/patchs/{3}Rn", num_folds=1)
processor.load_data(sample_size=sample_size, batch_size=batch_size)
loader = processor.loaders[0]
feature_extractor = FExtractor(dataloader=loader, use_8_bit=True)
features, labels, ids = feature_extractor.extract_features()
dataset = TensorDataset(torch.from_numpy(features), torch.from_numpy(labels), torch.from_numpy(ids))
f_test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

for i in range(it):

    processor = Loader(opt_root_dir=f"/home/stagiaire/D/D/patchs/{412}n", sar_root_dir=f"/home/stagiaire/D/D/patchs/{412}Rn", num_folds=1)
    processor.load_data(sample_size=sample_size, batch_size=batch_size)
    loader = processor.loaders[0]
    feature_extractor = FExtractor(dataloader=loader, use_8_bit=True)
    features, labels, ids = feature_extractor.extract_features()
    dataset = TensorDataset(torch.from_numpy(features), torch.from_numpy(labels), torch.from_numpy(ids))
    f_train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    head_i = copy.deepcopy(head)
        
    trainer = Trainer(head_i, f_train_loader)
    trainer.train(num_epochs=40)
        
    tester = Tester(head_i, f_test_loader, processor.combined_dataset)
    tester.evaluate(global_list)
    
    metrics.append(tester.report)
                
with open(f"/home/stagiaire/D/R/metrics/MV{it}3.pkl", 'wb') as f:
    pickle.dump(global_list, f)

Initializing SAR encoder
Initializing optical encoder
Initializing joint SAR-optical encoder


Extracting Features: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  3.87it/s]


Initializing SAR encoder
Initializing optical encoder
Initializing joint SAR-optical encoder


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:05<00:00,  7.17 epochs/s]



Jaccard index:  75.3%


                Predicted Class 0  Predicted Class 1
Actual Class 0                 98                 24
Actual Class 1                 12                110


              precision    recall  f1-score   support

           0       0.89      0.80      0.84       122
           1       0.82      0.90      0.86       122

    accuracy                           0.85       244
   macro avg       0.86      0.85      0.85       244
weighted avg       0.86      0.85      0.85       244


Initializing SAR encoder
Initializing optical encoder
Initializing joint SAR-optical encoder


...


Jaccard index:  66.9%


                Predicted Class 0  Predicted Class 1
Actual Class 0                 99                 23
Actual Class 1                 25                 97


              precision    recall  f1-score   support

           0       0.80      0.81      0.80       122
           1       0.81      0.80      0.80       122

    accuracy                           0.80       244
   macro avg       0.80      0.80      0.80       244
weighted avg       0.80      0.80      0.80       244


Initializing SAR encoder
Initializing optical encoder
Initializing joint SAR-optical encoder


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 24.78 epochs/s]



Jaccard index:  74.4%


                Predicted Class 0  Predicted Class 1
Actual Class 0                111                 11
Actual Class 1                 23                 99


              precision    recall  f1-score   support

           0       0.83      0.91      0.87       122
           1       0.90      0.81      0.85       122

    accuracy                           0.86       244
   macro avg       0.86      0.86      0.86       244
weighted avg       0.86      0.86      0.86       244


Initializing SAR encoder
Initializing optical encoder
Initializing joint SAR-optical encoder


Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 25.32 epochs/s]



Jaccard index:  80.6%


                Predicted Class 0  Predicted Class 1
Actual Class 0                110                 12
Actual Class 1                 14                108


              precision    recall  f1-score   support

           0       0.89      0.90      0.89       122
           1       0.90      0.89      0.89       122

    accuracy                           0.89       244
   macro avg       0.89      0.89      0.89       244
weighted avg       0.89      0.89      0.89       244




In [3]:
from R import RGenerator

report = RGenerator(metrics).report()

Precision (mean, stdev): 88%, 5%
Recall (mean, stdev): 86%, 7%
F1-score (mean, stdev): 86%, 4%
IoU (mean, stdev): 76%, 5%

[[107.  15.]
 [ 17. 105.]]

[[7. 7.]
 [8. 8.]]

